上位机代码的逻辑很简单：
所有的动作都有一个数字代号，如果想要物流车做出相应的动作，就用串口通信给下位机发送这个动作对应的数字。在下位机中有一个一直运行中的死循环接收函数，能接收到传输过来的数字，并操作物流车做出动作。因此，如果想要加入一个动作，只需要告知电控成员添加一个动作函数及对应的数字，在上位机中定义一个新的函数和选择分支，进行特定新动作编号的串口信息传输，就可以实现新动作的调用。

更改详细教程：
在config文件中，有所有的动作和数字的对应关系，是一个继承了枚举类型的类config，每一次调用moveControl类时都要传输一个config类的对象。而新增动作时，在config类下新增一个变量，写法如图：

In [ ]:
from enum import Enum


class Mode(Enum):
    x_distance = 1
    y_distance = 2
    rotate = 3
    qrcode = 4  # 通知下位机扫码
    x_dis_mm = 5
    y_dim_mm = 6
    grub_1 = 7  # 从原料区抓取物料块
    grub_2 = 8  # 从地面抓取物料块
    down_1 = 9  # 将物料块放到地面上
    down_2 = 10  # 精加工区，将物料块放到另一个物料块上
    adjust_height = 11  # 调整升降台到合适的位置    1.读取二维码  2.识别原料区  3.识别圆环   4.识别物料块圆心
    adjust_height_manual = 12
    mix_mode = 13  # 混动模式，一边底盘移动，一边移动升降台
    max_height = 48
    min_height = 49
    turn_front = 50
    back_home = 51
    get_qrPos = 54
    new_move = 14

其中，new_move是想要新增的动作，14则是想要执行新动作需要传输给下位机的代号。

而新增了这个动作与编号后，还要给主流程封装一个函数来方便调用。在整个项目中，已经封装好了一个函数def __send_serial_msg。这是一个move_Control类私有的函数，位于moveControl文件中的第31行左右。所有的串口信息都通过这个函数来发送。函数的第一个参数是上图中的Mode类的一个实例化对象，表示了想做的动作的编号。其他的参数则是某种动作所需要的变量，如移动的距离，旋转的角度，二维码的任务信息等等。每一个动作在这个函数中都有一个选择的分支，是通过if else来进行的选择跳转。新增的动作需要一个新的分支，即加入如图的代码片段：

In [ ]:
if mode == Mode.mix_mode
    self.self.send_buffer[1] = mode.value
elif mode == Mode.new_move:
    self.send_buffer[1] = mode.value

这一段代码中第一行明确了进入此分支的条件，第二行操作了要传输的串口信息，把第一位信息，即模式位信息初始化为新动作的对应编号。整个串口信息中，每一位所对应的含义在moveControl文件中第27/28行有对应的说明。

经过以上步骤，对下位机的信息传输函数就再次封装好了。而现在还需要给整体流程封装一个函数来调用这个动作，如下面的代码：

In [ ]:
def new_move(self):
    self.__send_serial_msg(mode=Mode.new_move)

在moveControl类中有很多公有函数可供整体流程中调用，如set_distance,rotate等。如果想要添加新动作，就在这几个函数后并列新增一个如上的新动作函数。并列添加的规则不必多说。到这为止，新动作就已经添加完毕了。现在可以在主流程中调用新动作:

In [ ]:
def run(self):
    self.moveControl.move_in_mm()

直接在move_2025文件中的类ComRun中的run函数调用这个新的函数即可。
这就完成了新增动作的所有操作步骤。